In [1]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
from python_module.pricing_model import SABRModel, BlackScholesModel

In [2]:
# -------
# CREATE CUSTOM FUNCTION
# -------
def compute_breakeven(market_data_df, step_per_year=250):
    
    model = LinearRegression(fit_intercept=False)
    model.fit(market_data_df[['gamma', 'vanna', 'volga']], -market_data_df['theta'])
    theta_gamma, theta_vanna, theta_volga = model.coef_

    daily_sigma = np.sqrt((theta_gamma*2) / (S0**2))
    ann_sigma = daily_sigma * np.sqrt(step_per_year)

    daily_vov = np.sqrt((theta_volga*2) / (ann_sigma**2))
    ann_vov = daily_vov * np.sqrt(step_per_year)

    corr = (theta_vanna)/(S0 * daily_sigma * ann_sigma * daily_vov)
    return ann_sigma, corr, ann_vov

In [3]:
# -------
# GENERATE MARKET DATA AND GREEKS
# -------

# Explicit pricing parameters
S0 = F = 100
option_type = 'put'
time_to_maturity = 365

# Hidden pricing parameters
alpha = +0.1
beta  = +1.0
rho   = -0.5
nu    = +1

# Pre-processing
T = time_to_maturity / 250

# Pricing factory
market_data_list = list()
for K in np.linspace(start=80, stop=100, num=20, dtype=int):

    option_type = 'call' if K > 100 else 'put'

    IV = SABRModel.compute_sigma(F, K, T, alpha, beta, rho, nu)
    
    pricing_results = BlackScholesModel.compute_option(S0, K, T, 0, IV, option_type, True)
    
    market_data_list.append({

        # Descriptive option features
        'symbol': f"{time_to_maturity}_{K}_{option_type}",
        'option_type': option_type, 
        'time_to_maturity': time_to_maturity, 

        # others features
        'S0': S0,
        'r': 0,

        # Pricing option features
        'F': F,
        'K': K,
        'T': T, 
        'IV': IV,

        # Greeks and option price
        **pricing_results})

market_data_df = pd.DataFrame(market_data_list)
market_data_df = market_data_df.set_index('symbol')
market_data_df['weights'] = 0
market_data_df.loc[market_data_df.index[-5:], 'weights'] = 2

In [4]:
market_data_df

,option_type,time_to_maturity,S0,r,F,K,T,IV,price,delta,gamma,vega,theta,vanna,volga,weights
symbol,,,,,,,,,,,,,,,,
365_80_put,put,365,100,0,100,80,1.46,0.179260,1.517656,-0.127454,0.009634,0.252128,-0.006191,-1.073118,147.624983,0
365_81_put,put,365,100,0,100,81,1.46,0.175170,1.594872,-0.135362,0.010277,0.262827,-0.006307,-1.104831,147.033590,0
365_82_put,put,365,100,0,100,82,1.46,0.171107,1.677121,-0.143837,0.010964,0.273909,-0.006420,-1.134711,145.778098,0
365_83_put,put,365,100,0,100,83,1.46,0.167070,1.764877,-0.152930,0.011699,0.285374,-0.006531,-1.162126,143.783188,0
365_84_put,put,365,100,0,100,84,1.46,0.163059,1.858675,-0.162695,0.012485,0.297215,-0.006639,-1.186323,140.970643,0
365_85_put,put,365,100,0,100,85,1.46,0.159076,1.959116,-0.173192,0.013323,0.309422,-0.006743,-1.206403,137.261027,0
365_86_put,put,365,100,0,100,86,1.46,0.155121,2.066889,-0.184488,0.014217,0.321977,-0.006842,-1.221302,132.576192,0
365_87_put,put,365,100,0,100,87,1.46,0.151196,2.182775,-0.196655,0.015169,0.334854,-0.006935,-1.229762,126.842940,0
365_88_put,put,365,100,0,100,88,1.46,0.147303,2.307666,-0.209773,0.016182,0.348013,-0.007022,-1.230302,119.998193,0


In [5]:
alpha_est, rho_est, nu_est = compute_breakeven(market_data_df)
alpha_est, rho_est, nu_est 

(0.10826351597269145, -0.14693857511377795, 0.9448267931884995)